We load our data.

In [1]:
from torch_geometric_temporal.dataset import WindmillOutputSmallDatasetLoader
#from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = WindmillOutputSmallDatasetLoader()
#loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

We look at our data.

**How many data points?**

17464

**What does a single datapoint look like?**

A single datapoint has a shape [11,8]

**What are the edge indices?**

The edge index is 2

**What are the edge attributes?**

The edge attributes are 121

**What kind of task is this - regression or classification?**

It is a regression task

In [10]:
print("Number of data points: ", dataset.snapshot_count)
snapshot = dataset[0]
print(snapshot)
print("Features: ", snapshot.num_features, "Edge index: ", len(snapshot.edge_index), "Edge attributes: ", len(snapshot.edge_attr))

Number of data points:  17464
Data(x=[11, 8], edge_index=[2, 121], edge_attr=[121], y=[11])
Features:  8 Edge index:  2 Edge attributes:  121


We split it into train and test. **Note need to be careful with temporal data.**

In [4]:
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

We define a model.

**What type of model?**

I used the GConvGRU model

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvGRU(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

We train the model.

**What optimizer?**

**How many epochs?**

In [8]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 8)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(25)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("MSE: {:.4f}".format(cost))

  4%|███▎                                                                                | 1/25 [00:21<08:37, 21.57s/it]

MSE: 1.0287


  8%|██████▋                                                                             | 2/25 [00:43<08:20, 21.74s/it]

MSE: 1.0144


 12%|██████████                                                                          | 3/25 [01:05<07:57, 21.69s/it]

...

MSE: 1.0028


 96%|███████████████████████████████████████████████████████████████████████████████▋   | 24/25 [08:50<00:21, 21.73s/it]

MSE: 1.0028


100%|███████████████████████████████████████████████████████████████████████████████████| 25/25 [09:12<00:00, 22.09s/it]

MSE: 1.0026


We evaluate.

**What is MSE?**

In [17]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9848


#### Using AdamW

In [14]:
from tqdm import tqdm

model1 = RecurrentGCN(node_features = 8)

optimizer = torch.optim.AdamW(model1.parameters(), lr=0.01)

model1.train()

for epoch in tqdm(range(25)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model1(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("MSE: {:.4f}".format(cost))

  4%|███▎                                                                                | 1/25 [00:22<09:07, 22.83s/it]

MSE: 1.0238


  8%|██████▋                                                                             | 2/25 [00:45<08:44, 22.80s/it]

MSE: 1.0146


 12%|██████████                                                                          | 3/25 [01:08<08:23, 22.91s/it]

...

MSE: 1.0020


 96%|███████████████████████████████████████████████████████████████████████████████▋   | 24/25 [09:06<00:22, 22.57s/it]

MSE: 1.0019


100%|███████████████████████████████████████████████████████████████████████████████████| 25/25 [09:28<00:00, 22.75s/it]

MSE: 1.0018


In [18]:
model1.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model1(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9826


#### Change epoches

In [20]:
from tqdm import tqdm

model2 = RecurrentGCN(node_features = 8)

optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)

model2.train()

for epoch in tqdm(range(20)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model2(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("MSE: {:.4f}".format(cost))

  5%|████▏                                                                               | 1/20 [00:22<07:00, 22.14s/it]

MSE: 1.0108


 10%|████████▍                                                                           | 2/20 [00:43<06:29, 21.63s/it]

MSE: 1.0080


 15%|████████████▌                                                                       | 3/20 [01:04<06:05, 21.51s/it]

...

MSE: 1.0011


 95%|██████████████████████████████████████████████████████████████████████████████▊    | 19/20 [06:46<00:21, 21.22s/it]

MSE: 1.0010


100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [07:07<00:00, 21.40s/it]

MSE: 1.0009


In [22]:
model2.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model2(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9841


#### more epoches

In [26]:
from tqdm import tqdm

model4 = RecurrentGCN(node_features = 8)

optimizer = torch.optim.Adam(model4.parameters(), lr=0.01)

model4.train()

for epoch in tqdm(range(30)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model4(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("MSE: {:.4f}".format(cost))

  3%|██▊                                                                                 | 1/30 [00:25<12:33, 25.99s/it]

MSE: 1.0084


  7%|█████▌                                                                              | 2/30 [00:47<10:52, 23.30s/it]

MSE: 1.0067


 10%|████████▍                                                                           | 3/30 [01:10<10:28, 23.28s/it]

...

MSE: 0.9983


 97%|████████████████████████████████████████████████████████████████████████████████▏  | 29/30 [11:50<00:25, 25.37s/it]

MSE: 0.9982


100%|███████████████████████████████████████████████████████████████████████████████████| 30/30 [12:14<00:00, 24.48s/it]

MSE: 0.9980


In [27]:
model4.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model4(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9871


#### SGD

In [23]:
from tqdm import tqdm

model3 = RecurrentGCN(node_features = 8)

optimizer = torch.optim.SGD(model3.parameters(), lr=0.01)

model3.train()

for epoch in tqdm(range(25)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model3(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("MSE: {:.4f}".format(cost))

  4%|███▎                                                                                | 1/25 [00:20<08:21, 20.89s/it]

MSE: 1.0284


  8%|██████▋                                                                             | 2/25 [00:41<08:01, 20.94s/it]

MSE: 1.0271


 12%|██████████                                                                          | 3/25 [01:03<07:47, 21.25s/it]

...

MSE: 1.0123


 96%|███████████████████████████████████████████████████████████████████████████████▋   | 24/25 [08:31<00:21, 21.16s/it]

MSE: 1.0120


100%|███████████████████████████████████████████████████████████████████████████████████| 25/25 [08:53<00:00, 21.32s/it]

MSE: 1.0116


In [24]:
model3.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model3(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9885
